In [ ]:
#!import "../0-setup/Packages.ipynb"

In [ ]:
using System;
using System.IO;
using System.Text.Json;
using System.Net.Http.Headers;
using System.Threading.Tasks;

class ConfigurationFile
{
    public string environment { get; set; }
    public Dictionary<string, Dictionary<string, string>> settings { get; set; } =
        new Dictionary<string, Dictionary<string, string>>();
}

// Load the configuration from `configuration.json`, if present.
var configurationFile = File.Exists("./configuration.json") ?
    JsonSerializer.Deserialize<ConfigurationFile>(File.ReadAllText("./configuration.json")) :
    new ConfigurationFile();

var environmentName = configurationFile.environment ?? "local";
var configuration = configurationFile.settings.ContainsKey(environmentName) ?
    configurationFile.settings[environmentName] :
    new Dictionary<string, string>();

var jinagaEndpoint = configuration.ContainsKey("jinagaEndpoint") ?
    configuration["jinagaEndpoint"] :
    "http://localhost:8000/jinaga";

var token = configuration.ContainsKey("token") ?
    configuration["token"] :
    null;


public class BearerAuthenticationHandler : Jinaga.Http.IHttpAuthenticationProvider
{
    public readonly string token;

    public BearerAuthenticationHandler(string token)
    {
        this.token = token;
    }

    public void SetRequestHeaders(HttpRequestHeaders headers)
    {
        headers.Authorization = new AuthenticationHeaderValue("Bearer", token);
    }

    public Task<bool> Reauthenticate()
    {
        return Task.FromResult(false);
    }
}

// Create a Jinaga client for the portal
var jinagaClient = JinagaClient.Create(opt =>
{
    opt.HttpEndpoint = new Uri(jinagaEndpoint);
    if (token != null)
    {
        opt.HttpAuthenticationProvider = new BearerAuthenticationHandler(token);
    }
});

// Log in to the portal.
var (user, profile) = await jinagaClient.Login();

(user, profile)